# Spatial joins

Goals of this notebook:

- Based on the `countries` and `cities` dataframes, determine for each city the country in which it is located.
- To solve this problem, we will use the the concept of a 'spatial join' operation: combining information of geospatial datasets based on their spatial relationship.

In [ ]:
%matplotlib inline

import pandas as pd
import geopandas

pd.options.display.max_rows = 10

In [ ]:
countries = geopandas.read_file("zip://./data/ne_110m_admin_0_countries.zip")
cities = geopandas.read_file("zip://./data/ne_110m_populated_places.zip")
rivers = geopandas.read_file("zip://./data/ne_50m_rivers_lake_centerlines.zip")

## Recap - joining dataframes

Pandas provides functionality to join or merge dataframes in different ways, see https://chrisalbon.com/python/data_wrangling/pandas_join_merge_dataframe/ for an overview and https://pandas.pydata.org/pandas-docs/stable/merging.html for the full documentation.

To illustrate the concept of joining the information of two dataframes with pandas, let's take a small subset of our `cities` and `countries` datasets: 

In [ ]:
cities2 = cities[cities['name'].isin(['Bern', 'Brussels', 'London', 'Paris'])].copy()
cities2['iso_a3'] = ['CHE', 'BEL', 'GBR', 'FRA']

In [ ]:
cities2

In [ ]:
countries2 = countries[['iso_a3', 'name', 'continent']]
countries2.head()

We added a 'iso_a3' column to the `cities` dataset, indicating a code of the country of the city. This country code is also present in the `countries` dataset, which allows us to merge those two dataframes based on the common column.

Joining the `cities` dataframe with `countries` will transfer extra information about the countries (the full name, the continent) to the `cities` dataframe, based on a common key:

In [ ]:
cities2.merge(countries2, on='iso_a3')

**But**, for this illustrative example, we added the common column manually, it is not present in the original dataset. However, we can still know how to join those two datasets based on their spatial coordinates.

## Recap - spatial relationships between objects

In the previous notebook [02-spatial-relationships.ipynb](./02-spatial-relationships-operations.ipynb), we have seen the notion of spatial relationships between geometry objects: within, contains, intersects, ...

In this case, we know that each of the cities is located *within* one of the countries, or the other way around that each country can *contain* multiple cities.

We can test such relationships using the methods we have seen in the previous notebook:

In [ ]:
france = countries.loc[countries['name'] == 'France', 'geometry'].squeeze()

In [ ]:
cities.within(france)

The above gives us a boolean series, indicating for each point in our `cities` dataframe whether it is located within the area of France or not.  
Because this is a boolean series as result, we can use it to filter the original dataframe to only show those cities that are actually within France:

In [ ]:
cities[cities.within(france)]

We could now repeat the above analysis for each of the countries, and add a column to the `cities` dataframe indicating this country. However, that would be tedious to do manually, and is also exactly what the spatial join operation provides us.

*(note: the above result is incorrect, but this is just because of the coarse-ness of the countries dataset)*

## Spatial join operation

<div class="alert alert-info" style="font-size:120%">
<b>SPATIAL JOIN</b> = *transferring attributes from one layer to another based on their spatial relationship* <br><br>


Different parts of this operations:

<ul>
  <li>The GeoDataFrame to which we want add information</li>
  <li>The GeoDataFrame that contains the information we want to add </li>
  <li>The spatial relationship we want to use to match both datasets ('intersects', 'contains', 'within')</li>
  <li>The type of join: left or inner join</li>
</ul>

</div>

In this case, we want to join the `cities` dataframe with the information of the `countries` dataframe, based on the spatial relationship between both datasets.

We use the [`geopandas.sjoin`](http://geopandas.readthedocs.io/en/latest/reference/geopandas.sjoin.html) function:

In [ ]:
joined = geopandas.sjoin(cities, countries, op='within', how='left')

In [ ]:
joined

In [ ]:
joined['continent'].value_counts()

---

**Exercise**: Which rivers pass within 1 degree of Paris? And for Hong Kong?

Tip: `rivers` contains empty geometries, make sure to get rid of them before anything else



<!--
paris = cities.query('name == "Paris"')\
              ['geometry']\
              .squeeze()\
              .buffer(1)

river_lines = rivers[rivers.geometry.notna()]

river_lines[river_lines.crosses(paris)]

hk = cities.query('name == "Hong Kong"')\
              ['geometry']\
              .squeeze()\
              .buffer(1)

river_lines = rivers[rivers.geometry.notna()]

river_lines[river_lines.crosses(hk)]
-->

---

**Exercise** [PRO]: Find all European cities within 1 degre of at least one river


This requires:

1. Obtaining a list of European cities
1. Creating a `GeoDataFrame` with all river lines
1. For every European city, checking whether at least one river "crosses" the area within 1 degree
1. If so, saving the record of a city

<!--
eu_cities = joined.query('continent == "Europe"')
river_lines = rivers[rivers.geometry.notna()]

answer = []
for id, city in eu_cities.iterrows():
    buf = city['geometry'].buffer(1)
    rivers_crossing = river_lines.crosses(buf).sum()
    if rivers_crossing > 0:
        answer.append(city)
answer = geopandas.GeoDataFrame(answer, 
                                crs=eu_cities.crs)
answer
-->

---

## The overlay operation

In the spatial join operation above, we are not changing the geometries itself. We are not joining geometries, but joining attributes based on a spatial relationship between the geometries. This also means that the geometries need to at least overlap partially.

If you want to create new geometries based on joining (combining) geometries of different dataframes into one new dataframe (eg by taking the intersection of the geometries), you want an **overlay** operation.

In [ ]:
africa = countries[countries['continent'] == 'Africa']

In [ ]:
africa.plot()

In [ ]:
cities['geometry'] = cities.buffer(2)

In [ ]:
geopandas.overlay(africa, cities, how='difference').plot()

<div class="alert alert-info" style="font-size:120%">
<b>REMEMBER</b> <br>

<ul>
  <li>**Spatial join**: transfer attributes from one dataframe to another based on the spatial relationship</li>
  <li>**Spatial overlay**: construct new geometries based on spatial operation between both dataframes (and combining attributes of both dataframes)</li>
</ul>

</div>

---

**Exercise**: if we define rural as those areas that are not within 50Km of the centre of a city, what is the extention of rural North America? What is the percentage that it covers?

Tip: make sure your projection allows you to calculate distances in metres ([wink wink](http://epsg.io/3083))

<!--
namerica = countries.query('continent == "North America"')\
                    .to_crs(epsg=3083)
urban = geopandas.GeoDataFrame(
            {'geometry':cities.to_crs(epsg=3083)\
                              .buffer(50000)}, \
            crs = namerica.crs)
        
rural = geopandas.overlay(namerica, urban, how='difference')

print('Rural: ', rural.unary_union.area * 1e-6, ' Km^2')
print('Rural is %.2f%% of the total extent'%
      (rural.unary_union.area * 100 / 
       namerica.unary_union.area))
-->

---